In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from models.slm import ScalableLM
from models.config import LlamaCLConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
dataset = load_dataset("LinhDuong/chatdoctor-200k", split='train')
model = ScalableLM(LlamaCLConfig())
model.load_state_dict(torch.load("/home/user/bhpeng/SLM-llama/outputs/medical/medical/checkpoint-4860/pytorch_model_retriever.bin"), strict=False)

/data/miniconda3/envs/alpaca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-15 19:59:47,081] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.56it/s]


<=============== Freeze Keys =============>


_IncompatibleKeys(missing_keys=['model.model.embed_tokens.weight', 'model.model.layers.0.self_attn.q_proj.weight', 'model.model.layers.0.self_attn.k_proj.weight', 'model.model.layers.0.self_attn.v_proj.weight', 'model.model.layers.0.self_attn.o_proj.weight', 'model.model.layers.0.self_attn.rotary_emb.inv_freq', 'model.model.layers.0.mlp.gate_proj.weight', 'model.model.layers.0.mlp.up_proj.weight', 'model.model.layers.0.mlp.down_proj.weight', 'model.model.layers.0.input_layernorm.weight', 'model.model.layers.0.post_attention_layernorm.weight', 'model.model.layers.1.self_attn.q_proj.weight', 'model.model.layers.1.self_attn.k_proj.weight', 'model.model.layers.1.self_attn.v_proj.weight', 'model.model.layers.1.self_attn.o_proj.weight', 'model.model.layers.1.self_attn.rotary_emb.inv_freq', 'model.model.layers.1.mlp.gate_proj.weight', 'model.model.layers.1.mlp.up_proj.weight', 'model.model.layers.1.mlp.down_proj.weight', 'model.model.layers.1.input_layernorm.weight', 'model.model.layers.1.pos

In [8]:
PROMPT_TEMPLATE = {
'with_sys': """[INST] <<SYS>>
{instruction}
<</SYS>>

{input} [/INST] \n""",

'without_sys': """[INST] {input} [/INST] \n"""
}

def prepare_data(example):
    raw_text = example['instruction'] + " ### " + example['input'] if 'instruction' in example.keys() else \
                   example['input']
    source = PROMPT_TEMPLATE['with_sys'].format_map(example) if 'instruction' in example.keys() else\
                 PROMPT_TEMPLATE['without_sys'].format_map(example)
    source = tokenizer(
            source,
            max_length=512,
            truncation=True,
            return_tensors='pt'
        )
    task = 'medical'
    return dict(
        raw_text=raw_text,
        input_ids=source['input_ids'].to("cuda"),
        attention_mask=source['attention_mask'].to("cuda"),
        task=task
    ), example['output'] + "</s>"

In [4]:
model = model.to("cuda")

In [17]:
import random
inps, outs = prepare_data(dataset[random.randint(0, 1000)])
out = model.generate(**inps)
print(outs, "\n")
print(tokenizer.decode(out[0]))

Skin is sensitive you should use non-toxic ointments creams and soaps it should be without proteins it should be not free milk free soda free you a use petroleum jelly for Chat Doctor. </s> 

<s> [INST] <<SYS>>
If you are a doctor, please answer the medical questions based on the patient's description.
<</SYS>>

my 1 year old daughter has red blotchy rash mostly on her stomach and chest . it appears mostly after baths and seems to be painful when inflamed. lotion or oil seems to make it worse. we have tried several different soaps - johnson s moisture therapy , aveeno, and aquaphor baby soaps. We have tried eucerin extreme dry skin lotion, aquaphor ointment, johnsons s creme lotion and regular lotion, and all cause the red raised rash to become even more inflamed and painful... please help! [/INST] 
It may be due to allergy to soap or body lotion. Stop using all these products. Use hypoallergenic soap like detail. Use lactocalamine lotion for bath. If not subsiding in 7 days consult de

: 